# Configuration

In [1]:
# ES connector config
es_user = 'admin'
es_pass = 'admin'
es_host = 'elasticsearch-1'
es_port = 9200
auth=(es_user, es_pass)

In [2]:
# samples postgres connector config
pg_user = 'test'
pg_pass = 'test'
pg_host = 'samples-db'
pg_port = 5432
pg_db_name = "db_samples"

# Init and import libraries

In [3]:
%matplotlib inline

# data wrangling + vis
import pandas as pd
import pandas.io.sql as psql
from matplotlib import pyplot as plt
import ssl
# databases

from opensearchpy import OpenSearch
import opensearchpy.helpers

# postgres
import psycopg2 as pg

# ms sql
import pyodbc


# Init connectors (Opensearch/OpenDistro version)

In [4]:

es = OpenSearch(
    hosts = [{'host': es_host, 'port': 9200}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    # client_cert = client_cert_path,
    # client_key = client_key_path,
    use_ssl = False,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

# check status
print('checking ES connection ...')
if not es.ping():
    raise Exception("Connection failed")
print('connected to ES')

checking ES connection ...


Exception: Connection failed

In [ ]:
# PostgreSQL connector
#
print('connecting to PostgreSQL ...')
pg_conn = pg.connect(database=pg_db_name,user=pg_user, password=pg_pass, host=pg_host, port=pg_port)

print('connected to PostgreSQL')

# Example queries

In [ ]:
def print_plot(cui_occ):
    cui_names = []
    cui_counts = []

    for cui, cnt in sorted(cui_occ.items(), key=lambda kv: kv[1], reverse=True):
        cui_names.append(cui)
        cui_counts.append(cnt)

    # display the results
    plt.figure(figsize=(25,8))

    plt.bar(range(len(cui_names)), list(cui_counts), align='center')
    plt.xticks(range(len(cui_occ)), list(cui_names), rotation=90)

    plt.tick_params(axis='x', which='major', labelsize=20)
    plt.tick_params(axis='y', which='major', labelsize=20)

    plt.show()

## ElasticSearch

### Direct data manipulations

In [ ]:
# example 1:
# retrieve the number of documents satisfying the query criteria
#
# an example query -- search for keyword 'cancer' in documents
query_body_text = {
    "query": {
        "match": {
            "document": "cancer"}
    }
}
index_to_query_text = 'medical_reports_text'

documents = es.search(index=index_to_query_text, body=query_body_text)

print(documents['hits']['total'])

In [ ]:
# example 2:
# - retrieve all the documents matching the query criteria 
# - calculate the CUI codes occurencies
# - visualize the results

query_body_medcat = {
    "query": {
        "match": {
            "nlp.source_value": "skin"}
    }
}
index_to_query_medcat = "medical_reports_anns_medcat_medmen_cancer"

# query the elasticsearch
results = opensearchpy.helpers.scan(es, index=index_to_query_medcat, query=query_body_medcat)

# calculate the occurencies
bio_cui_occ = {}
bio_tui_arr = {}
for item in results:
    cui = item['_source']['nlp.pretty_name']
    if cui not in bio_cui_occ: 
        bio_cui_occ[cui] = 1
    else:
        bio_cui_occ[cui] += 1
    
    tui = item['_source']['nlp.tui']
    if tui not in bio_tui_arr:
         bio_tui_arr[tui] = set()
    bio_tui_arr[tui].add(cui)
    

print_plot(bio_cui_occ)

In [ ]:
# example 2:
# - retrieve all the documents matching the query criteria 
# - calculate the CUI codes occurencies
# - visualize the results

def get_tui_cui_occ(es, index_name, cui_field_name, tui_field_name, query_field_name, query_phrase):
    # query the elasticsearch
    query = {
        "query": {
            "match": {
               query_field_name : query_phrase}
        }
    }
    results = opensearchpy.helpers.scan(es, index=index_name, query=query)

    # calculate the occurencies
    cui_occ = {}
    tui_arr = {}
    for item in results:
        #cui = (item['_source']['nlp.inst'], item['_source']['nlp.text'])
        cui = item['_source'][cui_field_name]
        if cui not in cui_occ: 
            cui_occ[cui] = 1
        else:
            cui_occ[cui] += 1

        tui = item['_source'][tui_field_name]
        if tui not in tui_arr:
             tui_arr[tui] = set()
        tui_arr[tui].add(cui)
    
    return (cui_occ, tui_arr)

In [ ]:
#bioyodie_results = get_tui_cui_occ(es, 'nifi_is_doc_gp_refletters_bioyodie', 'nlp.inst', 'nlp.TUI', 'nlp.text', 'cancer')
medcat_results = get_tui_cui_occ(es, 'medical_reports_anns_medcat_medmen_cancer', 'nlp.cui', 'nlp.tui', 'nlp.source_value', 'skin')

#print_plot(bioyodie_results[0])
print_plot(medcat_results[0])

### Using Pandas

In [17]:
# query template
#
query_body_medcat = {
    "query": {
        "match": {
            "nlp.source_value": "skin"}
    }
}

index_to_query_medcat = 'medical_reports_anns_medcat_medmen_cancer'


# need to re-query ES to fetch the results
#
results = opensearchpy.helpers.scan(es, index=index_to_query_medcat, query=query_body_medcat)


# create a data frame from the results
#
es_df = pd.DataFrame.from_dict([item['_source'] for item in results])
es_df.head()

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [medical_reports_anns_medcat_medmen_cancer]', medical_reports_anns_medcat_medmen_cancer, index_or_alias)

In [18]:
# select only a subset of columns and perform filtering based on TUI and CUI
es_df_sub = es_df.filter(items=['meta.docid', 'nlp.source_value', 'nlp.cui', 'nlp.tui'])
rows = es_df_sub.loc[(es_df_sub['nlp.tui'] == 'T191')]

print(rows[0:10])

NameError: name 'es_df' is not defined

## PostgreSQL (samples-db)

In [28]:
# example: read the DB and create pandas data frame
#
pg_df = pd.read_sql('SELECT docid, sampleid, dct FROM medical_reports_text', pg_conn)
pg_df.head()

,docid,sampleid,dct
0,1,1041,2019-10-30 16:53:58.476127
1,2,1042,2019-10-07 23:38:43.876148
2,3,1054,2019-12-02 17:05:18.100627
3,4,1068,2020-08-03 03:19:12.581134
4,5,1069,2020-03-14 06:47:10.383254
